In [4]:
# =========================
# CELL 1 — Install + Imports
# =========================
# If yfinance isn't installed in your Colab runtime, install it.
!pip -q install yfinance pandas

from datetime import datetime
import os
from pathlib import Path
import pandas as pd
import yfinance as yf


In [5]:
# =========================
# CELL 2 — Mount Google Drive
# =========================
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# === Cell 3: Create Drive folders + a versioned output folder ===

# Base folder in your Google Drive
BASE_DIR = Path("/content/drive/MyDrive/Financial Risk Dashboard 2025")
DATA_DIR = BASE_DIR / "data"

# Versioned subfolder so each run is preserved (timestamp)
run_stamp = datetime.now().strftime("%Y%m%d_%H%M%S")
RUN_DIR = DATA_DIR / f"run_{run_stamp}"
RUN_DIR.mkdir(parents=True, exist_ok=True)

# Log file for easy debugging
LOG_PATH = RUN_DIR / "download_log.txt"

print("Base folder:", BASE_DIR)
print("Data folder:", DATA_DIR)
print("This run output folder:", RUN_DIR)
print("Log file:", LOG_PATH)


Base folder: /content/drive/MyDrive/Financial Risk Dashboard 2025
Data folder: /content/drive/MyDrive/Financial Risk Dashboard 2025/data
This run output folder: /content/drive/MyDrive/Financial Risk Dashboard 2025/data/run_20251221_191436
Log file: /content/drive/MyDrive/Financial Risk Dashboard 2025/data/run_20251221_191436/download_log.txt


In [7]:
# === Cell 4: Settings (tickers + date range + columns) ===

# 10 tickers (portfolio)
TICKERS_CORE = ["AAPL","MSFT","NVDA","AMZN","META","JPM","XOM","JNJ","KO","TSLA"]

# Optional benchmarks (adds 2 files; comment out if you only want 10 files)
TICKERS_BENCH = ["SPY", "QQQ"]

# Combine list (10–15 files total depending on what you include)
TICKERS_ALL = TICKERS_CORE + TICKERS_BENCH

START_DATE = "2025-01-01"
# End date left as None so it fetches up to latest available
END_DATE = None

# What we keep for Power BI
KEEP_COLS = ["Date", "AdjClose", "Close", "Volume", "Ticker"]

print("Tickers:", TICKERS_ALL)
print("Start:", START_DATE, "| End: latest available")


Tickers: ['AAPL', 'MSFT', 'NVDA', 'AMZN', 'META', 'JPM', 'XOM', 'JNJ', 'KO', 'TSLA', 'SPY', 'QQQ']
Start: 2025-01-01 | End: latest available


In [10]:
import yfinance as yf

df_dbg = yf.download("AAPL", start="2025-01-01", interval="1d", auto_adjust=False, progress=False, threads=False)
print("Index name:", df_dbg.index.name)
print("Columns:", df_dbg.columns)
print(df_dbg.head(3))


Index name: Date
Columns: MultiIndex([('Adj Close', 'AAPL'),
            (    'Close', 'AAPL'),
            (     'High', 'AAPL'),
            (      'Low', 'AAPL'),
            (     'Open', 'AAPL'),
            (   'Volume', 'AAPL')],
           names=['Price', 'Ticker'])
Price        Adj Close       Close        High         Low        Open  \
Ticker            AAPL        AAPL        AAPL        AAPL        AAPL   
Date                                                                     
2025-01-02  242.752106  243.850006  249.100006  241.820007  248.929993   
2025-01-03  242.264297  243.360001  244.179993  241.889999  243.360001   
2025-01-06  243.896927  245.000000  247.330002  243.199997  244.309998   

Price         Volume  
Ticker          AAPL  
Date                  
2025-01-02  55740700  
2025-01-03  40244100  
2025-01-06  45045600  


In [11]:
# === Cell 5: Helper function to download one ticker + save CSV + log ===

# === Cell 5 (REPLACE): Helper function that handles MultiIndex columns from yfinance ===

def log(msg: str) -> None:
    with open(LOG_PATH, "a", encoding="utf-8") as f:
        f.write(msg + "\n")
    print(msg)

def _flatten_yf_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Flattens yfinance MultiIndex columns.
    Example: ('Adj Close','AAPL') -> 'Adj Close'
    """
    if isinstance(df.columns, pd.MultiIndex):
        # Keep the first level (Price) since you're downloading a single ticker per call
        df.columns = [c[0] for c in df.columns]
    return df

def download_ticker_2025(ticker: str) -> pd.DataFrame:
    try:
        log(f"\n=== {ticker} | START ===")

        df = yf.download(
            ticker,
            start=START_DATE,
            end=END_DATE,
            interval="1d",
            auto_adjust=False,
            progress=False,
            threads=False,
            group_by="column"  # helps but still may return MultiIndex
        )

        if df is None or df.empty:
            log(f"ERROR: {ticker} returned empty dataframe.")
            return pd.DataFrame()

        # Fix MultiIndex columns
        df = _flatten_yf_columns(df)

        # Reset index -> Date column
        df = df.reset_index()

        # Ensure Date column exists
        if "Date" not in df.columns:
            if "Datetime" in df.columns:
                df.rename(columns={"Datetime": "Date"}, inplace=True)
            elif "index" in df.columns:
                df.rename(columns={"index": "Date"}, inplace=True)
            else:
                raise KeyError(f"No Date column after reset_index(). Columns: {list(df.columns)}")

        # Standardize Adj Close naming
        if "Adj Close" in df.columns:
            df.rename(columns={"Adj Close": "AdjClose"}, inplace=True)
        if "AdjClose" not in df.columns:
            df["AdjClose"] = pd.NA

        # Ensure Close/Volume exist
        if "Close" not in df.columns:
            df["Close"] = pd.NA
        if "Volume" not in df.columns:
            df["Volume"] = pd.NA

        # Add ticker column
        df["Ticker"] = ticker

        # Keep only needed columns
        df_out = df[["Date", "AdjClose", "Close", "Volume", "Ticker"]].copy()

        # Clean + types
        df_out["Date"] = pd.to_datetime(df_out["Date"], errors="coerce").dt.date
        df_out = df_out.dropna(subset=["Date"])
        df_out = df_out.sort_values(["Date"]).drop_duplicates(subset=["Date", "Ticker"])

        # Save
        out_path = RUN_DIR / f"{ticker}_2025.csv"
        df_out.to_csv(out_path, index=False)

        # Summary
        first_date = df_out["Date"].min()
        last_date = df_out["Date"].max()
        rows = len(df_out)
        log(f"SAVED: {out_path.name} | rows={rows} | {first_date} -> {last_date}")
        log(f"=== {ticker} | OK ===")

        return df_out

    except Exception as e:
        log(f"EXCEPTION: {ticker} failed with error: {repr(e)}")
        return pd.DataFrame()



In [12]:
# === Cell 6: Download AAPL ===
aapl = download_ticker_2025("AAPL")
aapl.head()



=== AAPL | START ===
SAVED: AAPL_2025.csv | rows=243 | 2025-01-02 -> 2025-12-19
=== AAPL | OK ===


,Date,AdjClose,Close,Volume,Ticker
0,2025-01-02,242.752106,243.850006,55740700,AAPL
1,2025-01-03,242.264313,243.360001,40244100,AAPL
2,2025-01-06,243.896912,245.000000,45045600,AAPL
3,2025-01-07,241.119476,242.210007,40856000,AAPL
4,2025-01-08,241.607269,242.699997,37628900,AAPL


In [13]:
# === Cell 7: Download MSFT ===
msft = download_ticker_2025("MSFT")
msft.head()



=== MSFT | START ===
SAVED: MSFT_2025.csv | rows=243 | 2025-01-02 -> 2025-12-19
=== MSFT | OK ===


,Date,AdjClose,Close,Volume,Ticker
0,2025-01-02,415.514832,418.579987,16896500,MSFT
1,2025-01-03,420.249939,423.350006,16662900,MSFT
2,2025-01-06,424.716949,427.850006,20573600,MSFT
3,2025-01-07,419.277069,422.369995,18139100,MSFT
4,2025-01-08,421.451080,424.559998,15054600,MSFT


In [14]:
# === Cell 8: Download NVDA ===
nvda = download_ticker_2025("NVDA")
nvda.head()



=== NVDA | START ===
SAVED: NVDA_2025.csv | rows=243 | 2025-01-02 -> 2025-12-19
=== NVDA | OK ===


,Date,AdjClose,Close,Volume,Ticker
0,2025-01-02,138.272186,138.309998,198247200,NVDA
1,2025-01-03,144.430496,144.470001,229322500,NVDA
2,2025-01-06,149.389130,149.429993,265377400,NVDA
3,2025-01-07,140.101669,140.139999,351782200,NVDA
4,2025-01-08,140.071686,140.110001,227349900,NVDA


In [15]:
# === Cell 9: Download AMZN ===
amzn = download_ticker_2025("AMZN")
amzn.head()



=== AMZN | START ===
SAVED: AMZN_2025.csv | rows=243 | 2025-01-02 -> 2025-12-19
=== AMZN | OK ===


,Date,AdjClose,Close,Volume,Ticker
0,2025-01-02,220.220001,220.220001,33956600,AMZN
1,2025-01-03,224.190002,224.190002,27515600,AMZN
2,2025-01-06,227.610001,227.610001,31849800,AMZN
3,2025-01-07,222.110001,222.110001,28084200,AMZN
4,2025-01-08,222.130005,222.130005,25033300,AMZN


In [16]:
# === Cell 10: Download META ===
meta = download_ticker_2025("META")
meta.head()



=== META | START ===
SAVED: META_2025.csv | rows=243 | 2025-01-02 -> 2025-12-19
=== META | OK ===


,Date,AdjClose,Close,Volume,Ticker
0,2025-01-02,597.356384,599.239990,12682300,META
1,2025-01-03,602.729431,604.630005,11436800,META
2,2025-01-06,628.219055,630.200012,14560800,META
3,2025-01-07,615.947754,617.890015,12071500,META
4,2025-01-08,608.800232,610.719971,10085800,META


In [17]:
# === Cell 11: Download JPM ===
jpm = download_ticker_2025("JPM")
jpm.head()



=== JPM | START ===
SAVED: JPM_2025.csv | rows=243 | 2025-01-02 -> 2025-12-19
=== JPM | OK ===


,Date,AdjClose,Close,Volume,Ticker
0,2025-01-02,235.024765,240.000000,9220900,JPM
1,2025-01-03,238.236786,243.279999,9491100,JPM
2,2025-01-06,237.075287,240.850006,9917800,JPM
3,2025-01-07,239.358902,243.169998,8753400,JPM
4,2025-01-08,239.319550,243.130005,8675300,JPM


In [18]:
# === Cell 12: Download XOM ===
xom = download_ticker_2025("XOM")
xom.head()



=== XOM | START ===
SAVED: XOM_2025.csv | rows=243 | 2025-01-02 -> 2025-12-19
=== XOM | OK ===


,Date,AdjClose,Close,Volume,Ticker
0,2025-01-02,103.511009,107.309998,12685400,XOM
1,2025-01-03,104.041534,107.860001,14237900,XOM
2,2025-01-06,103.925781,107.739998,15623700,XOM
3,2025-01-07,104.900032,108.750000,12625900,XOM
4,2025-01-08,103.144463,106.930000,17858100,XOM


In [19]:
# === Cell 13: Download JNJ ===
jnj = download_ticker_2025("JNJ")
jnj.head()



=== JNJ | START ===
SAVED: JNJ_2025.csv | rows=243 | 2025-01-02 -> 2025-12-19
=== JNJ | OK ===


,Date,AdjClose,Close,Volume,Ticker
0,2025-01-02,139.742386,144.020004,6051300,JNJ
1,2025-01-03,139.907349,144.190002,5878800,JNJ
2,2025-01-06,139.393082,143.660004,7910500,JNJ
3,2025-01-07,141.886734,146.229996,8422000,JNJ
4,2025-01-08,138.044373,142.270004,11175600,JNJ


In [20]:
# === Cell 14: Download KO ===
ko = download_ticker_2025("KO")
ko.head()



=== KO | START ===
SAVED: KO_2025.csv | rows=243 | 2025-01-02 -> 2025-12-19
=== KO | OK ===


,Date,AdjClose,Close,Volume,Ticker
0,2025-01-02,60.067802,61.840000,12991000,KO
1,2025-01-03,59.980381,61.750000,10403200,KO
2,2025-01-06,59.067318,60.810001,17924200,KO
3,2025-01-07,59.096462,60.840000,17799600,KO
4,2025-01-08,59.941528,61.709999,14412400,KO


In [21]:
# === Cell 15: Download TSLA ===
tsla = download_ticker_2025("TSLA")
tsla.head()



=== TSLA | START ===
SAVED: TSLA_2025.csv | rows=243 | 2025-01-02 -> 2025-12-19
=== TSLA | OK ===


,Date,AdjClose,Close,Volume,Ticker
0,2025-01-02,379.279999,379.279999,109710700,TSLA
1,2025-01-03,410.440002,410.440002,95423300,TSLA
2,2025-01-06,411.049988,411.049988,85516500,TSLA
3,2025-01-07,394.359985,394.359985,75699500,TSLA
4,2025-01-08,394.940002,394.940002,73038800,TSLA


In [22]:
# === Cell 16: Download SPY (benchmark) ===
spy = download_ticker_2025("SPY")
spy.head()



=== SPY | START ===
SAVED: SPY_2025.csv | rows=243 | 2025-01-02 -> 2025-12-19
=== SPY | OK ===


,Date,AdjClose,Close,Volume,Ticker
0,2025-01-02,577.854126,584.640015,50204000,SPY
1,2025-01-03,585.079285,591.950012,37888500,SPY
2,2025-01-06,588.449707,595.359985,47679400,SPY
3,2025-01-07,581.797852,588.630005,60393100,SPY
4,2025-01-08,582.647888,589.489990,47304700,SPY


In [23]:
# === Cell 17: Download QQQ (benchmark) ===
qqq = download_ticker_2025("QQQ")
qqq.head()



=== QQQ | START ===
SAVED: QQQ_2025.csv | rows=243 | 2025-01-02 -> 2025-12-19
=== QQQ | OK ===


,Date,AdjClose,Close,Volume,Ticker
0,2025-01-02,508.309448,510.230011,36389800,QQQ
1,2025-01-03,516.627991,518.580017,29059500,QQQ
2,2025-01-06,522.565552,524.539978,36109700,QQQ
3,2025-01-07,513.240784,515.179993,36690300,QQQ
4,2025-01-08,513.330444,515.270020,30777800,QQQ


In [24]:
# === Cell 18: Verify outputs ===
created_files = sorted([p.name for p in RUN_DIR.glob("*.csv")])
print("CSV files created:", len(created_files))
for f in created_files:
    print(" -", f)

print("\nOpen the log file at:")
print(LOG_PATH)


CSV files created: 12
 - AAPL_2025.csv
 - AMZN_2025.csv
 - JNJ_2025.csv
 - JPM_2025.csv
 - KO_2025.csv
 - META_2025.csv
 - MSFT_2025.csv
 - NVDA_2025.csv
 - QQQ_2025.csv
 - SPY_2025.csv
 - TSLA_2025.csv
 - XOM_2025.csv

Open the log file at:
/content/drive/MyDrive/Financial Risk Dashboard 2025/data/run_20251221_191436/download_log.txt
